In [1]:
%load_ext autoreload
def get_gesture_counts(participant):
	gestures = {"drag":0,
				"draw":0,
				"ui":0,
				"tap":0,
				"doubletap":0,
				"tripletap":0,
				"hold":0,
				"pinch":0,
				"rev_pinch":0,
				"lasso":0,
				"box":0,
				"voice":0,
				"other":0}

	#Get all the events
	events = []
	for task in participant["tasks"].keys():
		events.extend(participant["tasks"][task])

	for event in events:
	 	if event["event_type"] == "tap":
	 		#Taps need special handling, as they might be double, triple, or hold
	 		if event["hold"]:
	 			gestures["hold"] += 1
	 		elif event["count"] == 2:
	 			gestures["doubletap"] += 1
	 		elif event["count"] == 3:
	 			gestures["tripletap"] += 1
	 		else:
	 			gestures["tap"] += 1
		elif event["event_type"] == "drag":
			#Drags might be drag or might be draw
			if event["draw"] is None:
				gestures["drag"] += 1
			else:
				gestures["draw"] += 1
		elif event["event_type"] == "pinch":
			#pinch can be pinch or reverse
			if event["reverse"]:
				gestures["rev_pinch"] += 1
			else:
				gestures["pinch"] += 1
		elif event["event_type"] == "voice_command":
			gestures["voice"] += 1
		elif event["event_type"] == "ui":
			gestures["ui"] += 1
		elif event["event_type"] == "memo":
			#Don't do anything with memos
			pass
		elif event["event_type"] == "lasso":
			gestures["lasso"] += 1
		elif event["event_type"] == "box_select":
			gestures["box"] += 1
		elif event["event_type"] == "other":
			gestures["other"] += 1		
		else:
			#This is an error, some event type wasn't handled
			print event["event_type"]

	return gestures

In [2]:
def average_counts(counts):
	participant_count = len(counts.keys())
	totals = {}
	#Collect the total counts
	for p in counts.keys():
		p_count = counts[p]
		for gesture in p_count.keys():
			if gesture in totals.keys():
				totals[gesture] += p_count[gesture]
			else:
				totals[gesture] = p_count[gesture]
	#Average across participants
	for gesture in totals.keys():
		totals[gesture] = totals[gesture]/float(participant_count)

	return totals

In [3]:
import all_data_handler
import pandas

In [4]:
adh = all_data_handler.UserData()

What I actually want to do here is have a dataframe where each user is also tagged with what condition they were in, so I can run ANOVA on that, with the condition as my categorical variable. 

In [5]:
data = []
for condition in adh.conditionMap.keys():
    counts = adh.applyCondition(get_gesture_counts, condition)
    #Convert to a list of dicts with user as a parameter of the dictionary
    #First put the user ID and the condition in the data
    for entry in counts:
        counts[entry]["user"] = entry
        counts[entry]["condition"] = condition
        #Tag as a multi-robot or more-or-less single robot condition
        if condition == "one" or condition == "unknown":
            counts[entry]["multi"] = False
        else:
            counts[entry]["multi"] = True
    data.extend(counts.values())

In [6]:
df = pandas.DataFrame(data)

In [7]:
df.set_index("user")

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,0,unknown,0,19,3,1,0,False,1,1,0,0,0,0,2
26,0,unknown,5,25,4,2,0,False,1,0,4,4,1,0,0
21,0,unknown,0,13,3,2,1,False,1,1,0,2,0,0,0
16,0,unknown,0,34,1,0,0,False,0,0,0,0,0,0,0
46,0,unknown,0,14,1,0,0,False,3,1,0,1,0,0,0
31,0,unknown,1,0,47,0,0,False,1,1,0,13,1,0,0
36,0,unknown,0,0,15,1,0,False,5,0,1,0,1,0,0
41,0,unknown,8,17,2,0,0,False,1,0,3,1,0,0,0
1,0,unknown,7,28,5,5,0,False,0,1,2,2,0,0,0


So that gets my data into a nice frame, now how do I tell jupyter to do ANOVA to it?

In [8]:
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [9]:
model = ols('drag ~ multi', data=df).fit()

In [10]:
table = sm.stats.anova_lm(model, typ=1)

In [11]:
table

,df,sum_sq,mean_sq,F,PR(>F)
multi,1,2096.163333,2096.163333,4.056319,0.049629
Residual,48,24804.716667,516.764931,NaN,NaN


I have two problems here. The first is that I'm not sure that I'm expressing the dependence between the condition and the variable correctly, and the second is that I don't know how to interpret the output. I think that a low PR(>F) is a good thing, but I'm not sure how low is good enough to say that a given gesture is related to the condition. 

In [12]:
# for colname in df.columns:
#     if colname != "multi" and colname != "condition":
#         model = ols('{0} ~ condition'.format(colname), data=df).fit()
#         table = sm.stats.anova_lm(model, typ=1)
#         print colname
#         print table
#         print

In [13]:
# for colname in df.columns:
#     if colname != "multi" and colname != "condition":
#         model = ols('{0} ~ multi'.format(colname), data=df).fit()
#         table = sm.stats.anova_lm(model, typ=1)
#         print colname
#         print table
#         print

In [14]:
df = pandas.DataFrame(data)

In [15]:
df.set_index('user')

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,0,unknown,0,19,3,1,0,False,1,1,0,0,0,0,2
26,0,unknown,5,25,4,2,0,False,1,0,4,4,1,0,0
21,0,unknown,0,13,3,2,1,False,1,1,0,2,0,0,0
16,0,unknown,0,34,1,0,0,False,0,0,0,0,0,0,0
46,0,unknown,0,14,1,0,0,False,3,1,0,1,0,0,0
31,0,unknown,1,0,47,0,0,False,1,1,0,13,1,0,0
36,0,unknown,0,0,15,1,0,False,5,0,1,0,1,0,0
41,0,unknown,8,17,2,0,0,False,1,0,3,1,0,0,0
1,0,unknown,7,28,5,5,0,False,0,1,2,2,0,0,0


In [16]:
model = ols('lasso ~ multi', data=df).fit()

In [17]:
table = sm.stats.anova_lm(model, typ=1)

In [18]:
table

,df,sum_sq,mean_sq,F,PR(>F)
multi,1,403.68,403.680000,6.110192,0.017038
Residual,48,3171.20,66.066667,NaN,NaN


In [19]:
model = ols('lasso ~ condition', data=df).fit()

In [20]:
table = sm.stats.anova_lm(model, typ=1)

In [21]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,803.68,200.920000,3.26263,0.019679
Residual,45,2771.20,61.582222,NaN,NaN


This makes it seem like whether the condition is a multi-robot condition is a stronger predictor (0.052 vs 0.163) of the use of lasso than what the actual condition is. Adding Dalton's coding dropped it to 0.0338 vs 0.062, which is great, as now the condition is becoming a stronger predictor in its own right, which supports the alternative hypothesis. 

In [22]:
df.corr(method='pearson')

,box,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
box,1.000000,-0.064274,-0.020765,-0.084332,0.275419,-0.051674,0.294605,-0.112812,-0.053877,-0.078905,0.137909,-0.125394,0.567690,-0.042780
doubletap,-0.064274,1.000000,-0.038902,-0.042275,0.355672,0.013880,-0.186544,-0.146372,0.026260,0.090368,0.315704,0.087537,0.009103,-0.102284
drag,-0.020765,-0.038902,1.000000,-0.198427,0.002558,0.165184,0.279145,-0.062095,0.262163,-0.184260,0.257071,0.181434,-0.106841,-0.103373
draw,-0.084332,-0.042275,-0.198427,1.000000,-0.090150,-0.003156,0.128975,-0.095541,-0.056593,0.018592,-0.029790,-0.047792,-0.070807,-0.070458
hold,0.275419,0.355672,0.002558,-0.090150,1.000000,0.189325,-0.037854,-0.261324,-0.151152,0.378577,0.332034,0.189951,0.252781,-0.161744
lasso,-0.051674,0.013880,0.165184,-0.003156,0.189325,1.000000,0.336038,-0.123726,0.302516,0.176826,0.045402,0.458766,-0.052469,-0.090731
multi,0.294605,-0.186544,0.279145,0.128975,-0.037854,0.336038,1.000000,0.321796,0.092519,0.027955,0.097767,0.022402,0.126063,0.119268
other,-0.112812,-0.146372,-0.062095,-0.095541,-0.261324,-0.123726,0.321796,1.000000,-0.060325,0.046164,-0.131933,-0.106003,-0.066646,-0.117997
pinch,-0.053877,0.026260,0.262163,-0.056593,-0.151152,0.302516,0.092519,-0.060325,1.000000,-0.090288,0.077835,-0.083657,-0.058025,-0.027575
rev_pinch,-0.078905,0.090368,-0.184260,0.018592,0.378577,0.176826,0.027955,0.046164,-0.090288,1.000000,-0.043993,-0.043994,-0.040243,-0.102119


Correlation shows that whether the condition is a multirobot condition correlates most strongly with box, drag, other, and lasso. I am not sure why drag and other got in there, although drag is also the most common gesture by far. I suspect that because there were a few multi-robot conditions that had very high counts of "other", but no single-robot conditions that have high counts of "other", the correlation isn't really representative of people's choices so much as it is a couple of outliers. 

Box and UI got a really strong correlation (0.73), which is probably because of box select and menu interactions being the main interaction method of RTS games. Adding whether a person plays RTS games to the data set would help confirm this, as box, ui, and RTS should all be highly correlated if that is the case. 

In [23]:
model = ols('ui ~ box', data=df).fit()

In [24]:
table = sm.stats.anova_lm(model, typ=1)

In [25]:
table

,df,sum_sq,mean_sq,F,PR(>F)
box,1,1039.366082,1039.366082,22.82488,0.000017
Residual,48,2185.753918,45.536540,NaN,NaN


This makes intuitive sense to me, as the use of box and UI are highly correlated, so one can predict the other.

In [26]:
model = ols('box ~ ui', data=df).fit()
table = sm.stats.anova_lm(model, typ=1)

In [27]:
table

,df,sum_sq,mean_sq,F,PR(>F)
ui,1,632.175328,632.175328,22.82488,0.000017
Residual,48,1329.444672,27.696764,NaN,NaN


Should switching those have changed the PR(>F) value? I don't suppose it should, correlation goes both ways. 

In [28]:
df_means = df.groupby('condition').mean()
df_means

,box,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
condition,,,,,,,,,,,,,,
hundred,6.8,0.9,19.6,9.7,1.2,5.5,True,6.6,0.0,0.4,8.5,0.0,1.9,2.3
one,0.0,3.6,10.5,4.8,1.7,0.3,False,0.1,0.0,0.5,10.6,0.4,2.0,0.7
ten,2.6,2.6,41.4,9.7,2.3,10.7,True,3.4,1.8,1.6,23.7,1.2,6.3,0.1
thousand,1.9,0.3,19.9,42.0,0.3,1.8,True,7.3,0.2,1.1,3.1,0.0,1.0,1.1
unknown,0.0,2.1,17.0,9.9,1.1,0.1,False,1.5,0.6,1.3,5.5,0.3,0.0,0.2


Mean gesture counts show some very clear peaks when means across users are taken within gestures, for example the thousand case having a peak in draws (although what causes that), the ten case having a peak in drags (10 case is low enough to still have single-robot moves, it's just inefficient). Lasso peaking in the 10 case and dropping off could be because thousand was viewed like unknown. I'm not sure what would account for the tap peak in 10 case, though. 

In [29]:
all_mean = df.mean()
all_mean

box           2.26
doubletap     1.90
drag         21.68
draw         15.22
hold          1.32
lasso         3.68
multi         0.60
other         3.78
pinch         0.52
rev_pinch     0.98
tap          10.28
tripletap     0.38
ui            2.24
voice         0.88
dtype: float64

In [30]:
import scipy.stats as stats

Scipy stats has f_oneway, so I can calculate the averages of each group, and then do that to see if the groups have the same population mean, although the means of means is probably not really what I want there. I could also get the totals for each group, but that also seems like a bad plan. 

In [31]:
all_counts = {}
for condition in adh.conditionMap.keys():
    counts = adh.applyCondition(get_gesture_counts, condition)
    avg = average_counts(counts)
    all_counts[condition] = avg

In [32]:
as_lists = {}
for condition in adh.conditionMap.keys():
    as_lists[condition] = [all_counts[condition][x] for x in sorted(all_counts[condition].keys())]

In [33]:
stats.f_oneway(as_lists['unknown'], as_lists['one'], as_lists['ten'], as_lists['hundred'], as_lists['thousand'])

F_onewayResult(statistic=0.96249639808037024, pvalue=0.43476332283556285)

That's not great, I want a very low P (e.g. less than 0.05). Check to see if unknown and one have different pop means (they shouldn't...)

In [34]:
stats.f_oneway(as_lists['unknown'], as_lists['one'])

F_onewayResult(statistic=0.037395814218522144, pvalue=0.84828867492621629)

In [35]:
for x in as_lists.keys():
    for y in as_lists.keys():
        if x != y:
            print x, y, stats.f_oneway(as_lists[x], as_lists[y])

unknown thousand F_onewayResult(statistic=0.73572641353507706, pvalue=0.39951825955964471)
unknown hundred F_onewayResult(statistic=0.76965267980279928, pvalue=0.38902226805088713)
unknown ten F_onewayResult(statistic=2.1420286996170432, pvalue=0.15628748090588765)
unknown one F_onewayResult(statistic=0.037395814218522144, pvalue=0.84828867492621629)
thousand unknown F_onewayResult(statistic=0.73572641353507728, pvalue=0.39951825955964471)
thousand hundred F_onewayResult(statistic=0.12033704504050748, pvalue=0.73169254699922881)
thousand ten F_onewayResult(statistic=0.20291057593943718, pvalue=0.65642470804350828)
thousand one F_onewayResult(statistic=0.96881775535144099, pvalue=0.33479465961369526)
hundred unknown F_onewayResult(statistic=0.7696526798027995, pvalue=0.38902226805088713)
hundred thousand F_onewayResult(statistic=0.12033704504050748, pvalue=0.73169254699922881)
hundred ten F_onewayResult(statistic=0.87305917952279344, pvalue=0.35941634946035028)
hundred one F_onewayResul

None of these appear to be very different. I could also normalize the data by dividing by the total gestures the user made, so rather than having a count, each user would have the proportion of their gestures that were a specific gesture. 

In [36]:
data

[{'box': 0,
  'condition': 'unknown',
  'doubletap': 0,
  'drag': 19,
  'draw': 3,
  'hold': 1,
  'lasso': 0,
  'multi': False,
  'other': 1,
  'pinch': 1,
  'rev_pinch': 0,
  'tap': 0,
  'tripletap': 0,
  'ui': 0,
  'user': u'11',
  'voice': 2},
 {'box': 0,
  'condition': 'unknown',
  'doubletap': 5,
  'drag': 25,
  'draw': 4,
  'hold': 2,
  'lasso': 0,
  'multi': False,
  'other': 1,
  'pinch': 0,
  'rev_pinch': 4,
  'tap': 4,
  'tripletap': 1,
  'ui': 0,
  'user': u'26',
  'voice': 0},
 {'box': 0,
  'condition': 'unknown',
  'doubletap': 0,
  'drag': 13,
  'draw': 3,
  'hold': 2,
  'lasso': 1,
  'multi': False,
  'other': 1,
  'pinch': 1,
  'rev_pinch': 0,
  'tap': 2,
  'tripletap': 0,
  'ui': 0,
  'user': u'21',
  'voice': 0},
 {'box': 0,
  'condition': 'unknown',
  'doubletap': 0,
  'drag': 34,
  'draw': 1,
  'hold': 0,
  'lasso': 0,
  'multi': False,
  'other': 0,
  'pinch': 0,
  'rev_pinch': 0,
  'tap': 0,
  'tripletap': 0,
  'ui': 0,
  'user': u'16',
  'voice': 0},
 {'box': 0,


In [37]:
normalized = []
for entry in data:
    total = 0
    for key in entry.keys():
        if key != "user" and key != "condition":
            total += entry[key]
    for key in entry.keys():
        if key != "user" and key != "condition":
            entry[key] = entry[key]/float(total)
    normalized.append(entry)

In [38]:
normalized

[{'box': 0.0,
  'condition': 'unknown',
  'doubletap': 0.0,
  'drag': 0.7037037037037037,
  'draw': 0.1111111111111111,
  'hold': 0.037037037037037035,
  'lasso': 0.0,
  'multi': 0.0,
  'other': 0.037037037037037035,
  'pinch': 0.037037037037037035,
  'rev_pinch': 0.0,
  'tap': 0.0,
  'tripletap': 0.0,
  'ui': 0.0,
  'user': u'11',
  'voice': 0.07407407407407407},
 {'box': 0.0,
  'condition': 'unknown',
  'doubletap': 0.10869565217391304,
  'drag': 0.5434782608695652,
  'draw': 0.08695652173913043,
  'hold': 0.043478260869565216,
  'lasso': 0.0,
  'multi': 0.0,
  'other': 0.021739130434782608,
  'pinch': 0.0,
  'rev_pinch': 0.08695652173913043,
  'tap': 0.08695652173913043,
  'tripletap': 0.021739130434782608,
  'ui': 0.0,
  'user': u'26',
  'voice': 0.0},
 {'box': 0.0,
  'condition': 'unknown',
  'doubletap': 0.0,
  'drag': 0.5652173913043478,
  'draw': 0.13043478260869565,
  'hold': 0.08695652173913043,
  'lasso': 0.043478260869565216,
  'multi': 0.0,
  'other': 0.043478260869565216,

In [39]:
normf = pandas.DataFrame(normalized)

In [40]:
normf.set_index('user')

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,0.000000,unknown,0.000000,0.703704,0.111111,0.037037,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.074074
26,0.000000,unknown,0.108696,0.543478,0.086957,0.043478,0.000000,0.000000,0.021739,0.000000,0.086957,0.086957,0.021739,0.000000,0.000000
21,0.000000,unknown,0.000000,0.565217,0.130435,0.086957,0.043478,0.000000,0.043478,0.043478,0.000000,0.086957,0.000000,0.000000,0.000000
16,0.000000,unknown,0.000000,0.971429,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
46,0.000000,unknown,0.000000,0.700000,0.050000,0.000000,0.000000,0.000000,0.150000,0.050000,0.000000,0.050000,0.000000,0.000000,0.000000
31,0.000000,unknown,0.015625,0.000000,0.734375,0.000000,0.000000,0.000000,0.015625,0.015625,0.000000,0.203125,0.015625,0.000000,0.000000
36,0.000000,unknown,0.000000,0.000000,0.652174,0.043478,0.000000,0.000000,0.217391,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000
41,0.000000,unknown,0.250000,0.531250,0.062500,0.000000,0.000000,0.000000,0.031250,0.000000,0.093750,0.031250,0.000000,0.000000,0.000000
1,0.000000,unknown,0.140000,0.560000,0.100000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,0.040000,0.000000,0.000000,0.000000


In [41]:
model = ols('lasso ~ condition', data=normf).fit()
table = sm.stats.anova_lm(model, typ=1)

In [42]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,0.066543,0.016636,2.183496,0.08606
Residual,45,0.342846,0.007619,NaN,NaN


In [43]:
model = ols('lasso ~ multi', data=normf).fit()
table = sm.stats.anova_lm(model, typ=1)

In [44]:
table

,df,sum_sq,mean_sq,F,PR(>F)
multi,1,0.000381,0.000381,0.044736,0.833386
Residual,48,0.409008,0.008521,NaN,NaN


In [45]:
normf_means = normf.groupby('condition').mean()

In [46]:
normf_means

,box,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
condition,,,,,,,,,,,,,,
hundred,0.107066,0.014663,0.291421,0.155247,0.018401,0.063635,0.018983,0.122398,0.000000,0.007666,0.103986,0.000000,0.020009,0.076525
one,0.000000,0.055251,0.437859,0.127583,0.048304,0.019841,0.000000,0.000943,0.000000,0.014053,0.210494,0.009826,0.051871,0.023974
ten,0.023033,0.026164,0.352892,0.139958,0.021489,0.106150,0.010784,0.048444,0.016036,0.018255,0.170396,0.011894,0.053872,0.000633
thousand,0.019000,0.008333,0.395794,0.247671,0.004632,0.028155,0.021291,0.158710,0.005757,0.019115,0.046574,0.000000,0.014864,0.030103
unknown,0.000000,0.051432,0.483824,0.219296,0.031095,0.004348,0.000000,0.054284,0.017930,0.030366,0.091934,0.008084,0.000000,0.007407


In [47]:
normf_means.loc['hundred'].values

array([ 0.1070662 ,  0.01466338,  0.29142064,  0.15524701,  0.01840061,
        0.06363522,  0.01898287,  0.12239802,  0.        ,  0.00766622,
        0.10398611,  0.        ,  0.02000853,  0.0765252 ])

In [48]:
stats.f_oneway(normf_means.loc['unknown'].values, normf_means.loc['one'].values, normf_means.loc['ten'].values, normf_means.loc['hundred'].values, normf_means.loc['thousand'].values)

F_onewayResult(statistic=9.0502172761941597e-32, pvalue=1.0)

In [49]:
stats.f_oneway(normf_means.loc['unknown'].values, normf_means.loc['one'].values)

F_onewayResult(statistic=3.9528787528516095e-32, pvalue=1.0)

In [50]:
stats.f_oneway(normf_means.loc['unknown'].values, normf_means.loc['hundred'].values)

F_onewayResult(statistic=-6.2223549527576116e-32, pvalue=nan)

This doesn't appear to be useful either, as this is saying that the population means are identical (or NaN for p values, which seems even less useful). This is probably because the normalization means that the mean for a user should be 1.0, so the mean for a population should be very close to 1.0 as well, and so now everything has a population mean of 1.0, and so of course they're not different. 

In [51]:
nonnorm_means = df.groupby('condition').mean()

In [52]:
nonnorm_means

,box,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
condition,,,,,,,,,,,,,,
hundred,6.8,0.9,19.6,9.7,1.2,5.5,True,6.6,0.0,0.4,8.5,0.0,1.9,2.3
one,0.0,3.6,10.5,4.8,1.7,0.3,False,0.1,0.0,0.5,10.6,0.4,2.0,0.7
ten,2.6,2.6,41.4,9.7,2.3,10.7,True,3.4,1.8,1.6,23.7,1.2,6.3,0.1
thousand,1.9,0.3,19.9,42.0,0.3,1.8,True,7.3,0.2,1.1,3.1,0.0,1.0,1.1
unknown,0.0,2.1,17.0,9.9,1.1,0.1,False,1.5,0.6,1.3,5.5,0.3,0.0,0.2


In [53]:
stats.f_oneway(nonnorm_means.loc['unknown'].values, nonnorm_means.loc['one'].values, nonnorm_means.loc['ten'].values, nonnorm_means.loc['hundred'].values, nonnorm_means.loc['thousand'].values)

F_onewayResult(statistic=0.97517385884898045, pvalue=0.4272921095118063)

In [54]:
for x in nonnorm_means.index:
    for y in nonnorm_means.index:
        if x != y:
            print x, y, stats.f_oneway(nonnorm_means.loc[x], nonnorm_means.loc[y])
    print

hundred one F_onewayResult(statistic=1.4022548781475184, pvalue=0.24705941709920107)
hundred ten F_onewayResult(statistic=0.85206421086557904, pvalue=0.36445656734128296)
hundred thousand F_onewayResult(statistic=0.11888118386050232, pvalue=0.7330247040366169)
hundred unknown F_onewayResult(statistic=0.81362776025236616, pvalue=0.37533052608111694)

one hundred F_onewayResult(statistic=1.4022548781475177, pvalue=0.24705941709920107)
one ten F_onewayResult(statistic=2.6170201255139562, pvalue=0.11779227423700038)
one thousand F_onewayResult(statistic=1.0020858015563405, pvalue=0.32603238723104405)
one unknown F_onewayResult(statistic=0.036442194640240448, pvalue=0.85008804739360633)

ten hundred F_onewayResult(statistic=0.8520642108655786, pvalue=0.36445656734128296)
ten one F_onewayResult(statistic=2.6170201255139562, pvalue=0.11779227423700038)
ten thousand F_onewayResult(statistic=0.19981083250077794, pvalue=0.65857510864211088)
ten unknown F_onewayResult(statistic=2.1561382681857872

hundred one F_onewayResult(statistic=3.0445401036633575, pvalue=0.093803195615485752)  
hundred ten F_onewayResult(statistic=0.62995257328961762, pvalue=0.43515030889820316)  
hundred thousand F_onewayResult(statistic=0.74646064762372855, pvalue=0.39615212673610622)  
hundred unknown F_onewayResult(statistic=0.81132768286727652, pvalue=0.37667821218931918)  

one hundred F_onewayResult(statistic=3.0445401036633579, pvalue=0.093803195615485752)  
one ten F_onewayResult(statistic=2.8456501473790681, pvalue=0.10457865347925255)
one thousand F_onewayResult(statistic=0.6589196563127182, pvalue=0.42491972670585898)  
one unknown F_onewayResult(statistic=0.55200384002671354, pvalue=0.46471106985119826)  

ten hundred F_onewayResult(statistic=0.62995257328961785, pvalue=0.43515030889820316)  
ten one F_onewayResult(statistic=2.8456501473790681, pvalue=0.10457865347925255)  
ten thousand F_onewayResult(statistic=1.6498127555285791, pvalue=0.21124481628590228)  
ten unknown F_onewayResult(statistic=1.7036814881039264, pvalue=0.20418288297121795)  

thousand hundred F_onewayResult(statistic=0.74646064762372843, pvalue=0.39615212673610622)  
thousand one F_onewayResult(statistic=0.6589196563127182, pvalue=0.42491972670585898)  
thousand ten F_onewayResult(statistic=1.6498127555285791, pvalue=0.21124481628590228)  
thousand unknown F_onewayResult(statistic=0.002188696239585675, pvalue=0.96307289865537826)  

unknown hundred F_onewayResult(statistic=0.81132768286727719, pvalue=0.37667821218931918)  
unknown one F_onewayResult(statistic=0.55200384002671332, pvalue=0.46471106985119826)  
unknown ten F_onewayResult(statistic=1.7036814881039262, pvalue=0.20418288297121795)  
unknown thousand F_onewayResult(statistic=0.0021886962395856746, pvalue=0.96307289865537826)  

This is the values from the averages that I calculated with python, before adding Dalton's codes for 31-40, it seems pretty close to what pandas came up with (although not identical). Also, adding Dalton's codes affected the P-values, although some went up and some went down, which is not what I'd really like to see (all of them going down).  

In [55]:
groups = df.groupby('condition').groups

In [56]:
tens = df.loc[groups['ten']].drop(['condition', 'multi'],axis=1)

In [57]:
unknowns = df.loc[groups['unknown']].drop(['condition', 'multi'],axis=1)

In [58]:
thousands = df.loc[groups['thousand']].drop(['condition', 'multi'],axis=1)

In [59]:
ones = df.loc[groups['one']].drop(['condition', 'multi'],axis=1)

In [60]:
hundreds = df.loc[groups['hundred']].drop(['condition', 'multi'],axis=1)

In [61]:
stats.f_oneway(unknowns, ones, tens, hundreds, thousands)

F_onewayResult(statistic=array([ 2.11739051,  0.98810872,  2.84689215,  0.89538737,  1.95899054,
        3.26262991,  1.78938085,  1.55434783,  0.45806329,  1.96410962,
        2.17365269,  0.8724858 ,  0.10909091,  0.8874502 ]), pvalue=array([ 0.0942383 ,  0.42373449,  0.03463158,  0.47462886,  0.11709708,
        0.01967861,  0.14761274,  0.20288588,  0.76602226,  0.11627936,
        0.08723145,  0.48788519,  0.97873643,  0.47919268]))

Again, this makes it look like there isn't a statistically significant variation in the data on each class. 

In [62]:
stats.f_oneway(unknowns['lasso'].values, ones['lasso'].values, tens['lasso'].values, hundreds['lasso'].values, thousands['lasso'].values)

F_onewayResult(statistic=3.2626299076212466, pvalue=0.019678614271677039)

In [63]:
groups = df.groupby('multi').groups

In [64]:
multis = df.loc[groups[True]].drop(['condition','multi'], axis = 1)
singles = df.loc[groups[False]].drop(['condition','multi'], axis = 1)

In [65]:
stats.f_oneway(multis, singles)

F_onewayResult(statistic=array([ 4.56196945,  1.73055028,  4.05631886,  0.81196764,  0.06887892,
        6.11019173,  5.54469087,  0.41441313,  0.03754124,  0.46323319,
        0.02410042,  0.77513023,  0.34824667,  0.69264448]), pvalue=array([ 0.03781789,  0.19458984,  0.04962947,  0.37203831,  0.79409925,
        0.01703818,  0.02267441,  0.52280214,  0.84718486,  0.4993869 ,
        0.87728067,  0.38302162,  0.55787541,  0.40938799]))

In [66]:
stats.f_oneway(multis['lasso'].values, singles['lasso'].values)

F_onewayResult(statistic=6.1101917255297682, pvalue=0.017038182003712366)

Now we're getting somewhere, one of the statistics dropped into statistical relevance (0.03 < 0.05) so for that case at least, the null hypothesis has a 95% chance of being wrong. 

In [67]:
df.groupby('condition').describe()

box  doubletap        drag        draw       hold  \
condition                                                                  
hundred   count  10.000000  10.000000   10.000000   10.000000  10.000000   
          mean    6.800000   0.900000   19.600000    9.700000   1.200000   
          std     9.507891   1.449138   20.056587   10.698390   1.475730   
          min     0.000000   0.000000    0.000000    0.000000   0.000000   
          25%     0.000000   0.000000    5.250000    1.250000   0.000000   
          50%     0.000000   0.000000   16.000000    6.000000   0.500000   
          75%    13.500000   1.000000   23.250000   17.750000   2.000000   
          max    23.000000   4.000000   58.000000   30.000000   4.000000   
one       count  10.000000  10.000000   10.000000   10.000000  10.000000   
          mean    0.000000   3.600000   10.500000    4.800000   1.700000   
          std     0.000000   8.289217    5.892557    5.902918   1.567021   
          min     0.000000   0.000000    0.000000    0.000000   0.000000   
          25%     0.000000   0.000000   10.000000    0.250000   0.250000   
          50%     0.000000   1.000000   11.000000    2.500000   1.500000   
          75%     0.000000   2.000000   14.000000    6.000000   2.750000   
          max     0.000000  27.000000   17.000000   17.000000   4.000000   
ten       count  10.000000  10.000000   10.000000   10.000000  10.000000   
          mean    2.600000   2.600000   41.400000    9.700000   2.300000   
          std     7.545418   2.458545   38.491558   15.485118   2.540779   
          min     0.000000   0.000000    0.000000    0.000000   0.000000   
          25%     0.000000   1.000000   12.750000    0.250000   0.000000   
          50%     0.000000   2.000000   33.000000    2.000000   1.500000   
          75%     0.000000   4.000000   64.250000    8.750000   4.000000   
          max    24.000000   8.000000  112.000000   39.000000   7.000000   
thousand  count  10.000000  10.000000   10.000000   10.000000  10.000000   
          mean    1.900000   0.300000   19.900000   42.000000   0.300000   
          std     6.008328   0.948683   18.591814  110.362031   0.674949   
          min     0.000000   0.000000    0.000000    0.000000   0.000000   
          25%     0.000000   0.000000    5.250000    1.250000   0.000000   
          50%     0.000000   0.000000   17.500000    4.000000   0.000000   
          75%     0.000000   0.000000   27.250000   17.000000   0.000000   
          max    19.000000   3.000000   60.000000  355.000000   2.000000   
unknown   count  10.000000  10.000000   10.000000   10.000000  10.000000   
          mean    0.000000   2.100000   17.000000    9.900000   1.100000   
          std     0.000000   3.247221   11.005049   14.309670   1.595131   
          min     0.000000   0.000000    0.000000    1.000000   0.000000   
          25%     0.000000   0.000000   13.250000    2.250000   0.000000   
          50%     0.000000   0.000000   18.000000    3.500000   0.500000   
          75%     0.000000   4.000000   23.750000   12.500000   1.750000   
          max     0.000000   8.000000   34.000000   47.000000   5.000000   

                     lasso  multi      other      pinch  rev_pinch  \
condition                                                            
hundred   count  10.000000     10  10.000000  10.000000  10.000000   
          mean    5.500000      1   6.600000   0.000000   0.400000   
          std     9.891073      0   9.335714   0.000000   0.516398   
          min     0.000000   True   0.000000   0.000000   0.000000   
          25%     0.000000      1   0.000000   0.000000   0.000000   
          50%     0.000000      1   2.000000   0.000000   0.000000   
          75%     5.750000      1  12.000000   0.000000   1.000000   
          max    29.000000   True  28.000000   0.000000   1.000000   
one       count  10.000000     10  10.000000  10.000000  10.000000   
          mean    0.300000      0   0.100000   0.000000   0.

The standard deviations are all over the place, ANOVA expects groups to have similar standard deviations. There is probably some normalization method to account for this. 

In [68]:
for col in df.columns:
    if col != 'condition' and col != 'multi':
        print col, stats.shapiro(df[col])

box (0.40059953927993774, 5.367310328824315e-13)
doubletap (0.47457683086395264, 3.964613359830338e-12)
drag (0.7945889234542847, 6.594880233024014e-07)
draw (0.2696825861930847, 2.313313386334559e-14)
hold (0.7716935276985168, 2.0717934035019425e-07)
lasso (0.49883705377578735, 7.973714570563839e-12)
other (0.5475205183029175, 3.490268837835764e-11)
pinch (0.27908456325531006, 2.858397044991183e-14)
rev_pinch (0.4623509645462036, 2.811747145878285e-12)
tap (0.5695058107376099, 7.048046174462641e-11)
tripletap (0.39901965856552124, 5.152989580176071e-13)
ui (0.2933441996574402, 3.9551342886565144e-14)
user (0.1433510184288025, 1.6070104951677639e-15)
voice (0.33181965351104736, 9.733415516719607e-14)


The _really tiny_ p values on all of these seems to indicate that my data is sampled from a normal distribution, which is good for ANOVA. I probably only have to worry about fixing the standard deviations, not normalizing the distribution. 

In [69]:
groups = df.groupby('condition').groups
for group in groups:
    g = df.loc[groups[group]]
    print group
    for col in g.columns:
        if col != 'condition' and col != 'multi':
            print col, stats.shapiro(g[col])

unknown
box (1.0, 1.0)
doubletap (0.6891496181488037, 0.0006509747472591698)
drag (0.9457173347473145, 0.6181904077529907)
draw (0.6668258309364319, 0.00034990455606020987)
hold (0.7405534982681274, 0.0027384324930608273)
lasso (0.36572057008743286, 1.0036932707180313e-07)
other (0.807977557182312, 0.018121767789125443)
pinch (0.6404852867126465, 0.00016867939848452806)
rev_pinch (0.8021806478500366, 0.01541060023009777)
tap (0.6105780601501465, 7.395610737148672e-05)
tripletap (0.5941736698150635, 4.713490488938987e-05)
ui (1.0, 1.0)
user (0.2300717830657959, 2.8309188326858248e-09)
voice (0.36572057008743286, 1.0036932707180313e-07)
thousand
box (0.36572057008743286, 1.0036932707180313e-07)
doubletap (0.36572057008743286, 1.0036932707180313e-07)
drag (0.9118657112121582, 0.2940542697906494)
draw (0.42908787727355957, 5.416909516497981e-07)
hold (0.5316476821899414, 8.56427777762292e-06)
lasso (0.6522319316864014, 0.00023345813679043204)
other (0.6308025121688843, 0.000129099833429791

/usr/lib/python2.7/dist-packages/scipy/stats/morestats.py:1326: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


The normality test mostly stays small when checking within groups, but not for all gestures within a group. Of course, if a gesture is never used, all the counts will be 0, and I suspect that the normality test is not well-behaved in that degnerate case. 

In [70]:
groups = df.groupby('condition').groups
znorm = []
for group in groups:
    g = df.loc[groups[group]]
    means = g.mean(numeric_only=True)
    std_devs = g.std(numeric_only=True)
    #print means['box'], std_devs['box']
    #For each row in the group
    gdata = []
    for row in g.iterrows():
        #For each value in the row, calculate its zscore
        zscores = {}
        for index in df.axes[1].tolist():
            if index != 'condition' and index != "multi" and index != "user":
                if std_devs[index] != 0:
                    zscores[index] = (row[1][index] - means[index])/std_devs[index]   
                else: 
                    zscores[index] = 0.0
        #Put the user ID back in
        zscores['user'] = row[1]['user']
        zscores['condition'] = row[1]['condition']
        znorm.append(zscores)
        
    

This is an attempt to z-score the data within each gesture and within each group, so taking the mean and standard deviation of (for example) "drag" in the 10-robot case, and then using those values to z-score each user in the 10-robot case. 

It would also be possible to take the mean and std. dev. of all of a user's gestures, and then z-score all of their gestures, so that "gestures used" would have a mean of 0 and std. dev. of 1. I'd expect that to have a similar result to the attempt to normalize gestures by dividing by total gesture count per user, where the counts would get turned into a proportion.

Zscore is a case of standardization (0 mean and unit variance), which I don't think is really what I'm going for. 

In [71]:
znorm_df = pandas.DataFrame(znorm)

In [72]:
znorm_df.set_index('user')

,box,condition,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,
11,0.000000,unknown,-0.646707,0.181735,-0.482191,-0.062691,-0.316228,-0.331295,0.774597,-0.829599,-0.545027,-0.621059,0.000000,2.846050
26,0.000000,unknown,0.893071,0.726939,-0.412309,0.564217,-0.316228,-0.331295,-1.161895,1.723014,-0.148644,1.449138,0.000000,-0.316228
21,0.000000,unknown,-0.646707,-0.363470,-0.482191,0.564217,2.846050,-0.331295,0.774597,-0.829599,-0.346835,-0.621059,0.000000,-0.316228
16,0.000000,unknown,-0.646707,1.544745,-0.621957,-0.689598,-0.316228,-0.993884,-1.161895,-0.829599,-0.545027,-0.621059,0.000000,-0.316228
46,0.000000,unknown,-0.646707,-0.272602,-0.621957,-0.689598,-0.316228,0.993884,0.774597,-0.829599,-0.445931,-0.621059,0.000000,-0.316228
31,0.000000,unknown,-0.338751,-1.544745,2.592652,-0.689598,-0.316228,-0.331295,0.774597,-0.829599,0.743218,1.449138,0.000000,-0.316228
36,0.000000,unknown,-0.646707,-1.544745,0.356402,-0.062691,-0.316228,2.319062,-1.161895,-0.191446,-0.545027,1.449138,0.000000,-0.316228
41,0.000000,unknown,1.816938,0.000000,-0.552074,-0.689598,-0.316228,-0.331295,-1.161895,1.084861,-0.445931,-0.621059,0.000000,-0.316228
1,0.000000,unknown,1.508983,0.999541,-0.342426,2.444940,-0.316228,-0.993884,0.774597,0.446707,-0.346835,-0.621059,0.000000,-0.316228


In [73]:
model = ols('lasso ~ condition', data=znorm_df).fit()
table = sm.stats.anova_lm(model, typ=1)

In [74]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,4.011519e-30,1.002880e-30,1.002880e-30,1
Residual,45,4.500000e+01,1.000000e+00,NaN,NaN


In [75]:
groups = znorm_df.groupby('condition').groups

In [76]:
unknowns = znorm_df.loc[groups['unknown']].drop(['condition'],axis=1)

In [77]:
tens = znorm_df.loc[groups['ten']].drop(['condition'],axis=1)

In [78]:
stats.f_oneway(unknowns, tens)

F_onewayResult(statistic=array([  4.26325641e-14,   1.42108547e-14,  -7.10542736e-15,
        -7.10542736e-15,  -7.10542736e-15,   1.42108547e-14,
         2.13162821e-14,   1.42108547e-14,  -2.13162821e-14,
        -2.13162821e-14,  -7.10542736e-15,  -1.42108547e-14,
         8.72727273e-02,   0.00000000e+00]), pvalue=array([ 0.99999984,  0.99999991,         nan,         nan,         nan,
        0.99999991,  0.99999988,  0.99999991,         nan,         nan,
               nan,         nan,  0.77105406,  1.        ]))

Clearly, Z-scores were not the way to go here. What I would want to see is lower p-values, these are nearly one. These are normalized with the means and standard deviations of COLUMNS, because I want to be able to compare the same gestures across users, and normalizing across all the gestures that a user did seems like the wrong thing to do. 

In [79]:
stats.f_oneway(unknowns['lasso'].values, tens['lasso'].values)

F_onewayResult(statistic=4.3140830754274072e-32, pvalue=1.0)

In [80]:
df.set_index('user')
row_stddevs = df.std(axis=1, numeric_only=True)
row_means = df.mean(axis=1, numeric_only=True)

In [81]:
rownormed = []
for row in df.iterrows():
    rowdata = {}
    for index in df.axes[1].tolist():
        if index == 'user' or index == 'condition' or index == 'multi':
            rowdata[index] = row[1][index]
        else:
            if row_stddevs[row[0]] == 0:
                rowdata[index] = 0
            else:
                rowdata[index] = (row[1][index]-row_means[row[0]])/row_stddevs[row[0]]
    rownormed.append(rowdata)

In [82]:
rownormed_df = pandas.DataFrame(rownormed)
rownormed_df.set_index('user')

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,-0.385757,unknown,-0.385757,3.414661,0.214309,-0.185735,-0.385757,False,-0.185735,-0.185735,-0.385757,-0.385757,-0.385757,-0.385757,0.014287
26,-0.503843,unknown,0.262874,3.329743,0.109531,-0.197156,-0.503843,False,-0.350499,-0.503843,0.109531,0.109531,-0.350499,-0.503843,-0.503843
21,-0.481698,unknown,-0.481698,3.330001,0.397925,0.104717,-0.188491,False,-0.188491,-0.188491,-0.481698,0.104717,-0.481698,-0.481698,-0.481698
16,-0.275627,unknown,-0.275627,3.472896,-0.165376,-0.275627,-0.275627,False,-0.275627,-0.275627,-0.275627,-0.275627,-0.275627,-0.275627,-0.275627
46,-0.384528,unknown,-0.384528,3.383845,-0.115358,-0.384528,-0.384528,False,0.422981,-0.115358,-0.384528,-0.115358,-0.384528,-0.384528,-0.384528
31,-0.360587,unknown,-0.281708,-0.360587,3.346696,-0.360587,-0.360587,False,-0.281708,-0.281708,-0.360587,0.664832,-0.281708,-0.360587,-0.360587
36,-0.403772,unknown,-0.403772,-0.403772,3.282840,-0.157998,-0.403772,False,0.825099,-0.403772,-0.157998,-0.403772,-0.157998,-0.403772,-0.403772
41,-0.479941,unknown,1.199852,3.089620,-0.059993,-0.479941,-0.479941,False,-0.269967,-0.479941,0.149982,-0.269967,-0.479941,-0.479941,-0.479941
1,-0.482101,unknown,0.462817,3.297574,0.192841,0.192841,-0.482101,False,-0.482101,-0.347113,-0.212125,-0.212125,-0.482101,-0.482101,-0.482101


In [83]:
model = ols('lasso ~ condition', data=znorm_df).fit()
table = sm.stats.anova_lm(model, typ=1)

In [84]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,4.011519e-30,1.002880e-30,1.002880e-30,1
Residual,45,4.500000e+01,1.000000e+00,NaN,NaN


I imported 10 of Dalton's codings, the other ten are not yet available. I'm not great at stats, but what I've learned so far is that if you just attempt to norm things in a kind of ham-fisted groping-in-the-dark way, you're going to have a bad time. 

In [85]:
data = []
for condition in adh.conditionMap.keys():
    counts = adh.applyCondition(get_gesture_counts, condition)
    #Convert to a list of dicts with user as a parameter of the dictionary
    #First put the user ID and the condition in the data
    for entry in counts:
        counts[entry]["user"] = entry
        counts[entry]["condition"] = condition
        #Tag as a multi-robot or more-or-less single robot condition
        #if condition == "one" or condition == "unknown":
        #    counts[entry]["multi"] = False
        #else:
        #    counts[entry]["multi"] = True
    data.extend(counts.values())
raw_data = pandas.DataFrame(data)
raw_data.set_index("user")
    

,box,condition,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,
11,0,unknown,0,19,3,1,0,1,1,0,0,0,0,2
26,0,unknown,5,25,4,2,0,1,0,4,4,1,0,0
21,0,unknown,0,13,3,2,1,1,1,0,2,0,0,0
16,0,unknown,0,34,1,0,0,0,0,0,0,0,0,0
46,0,unknown,0,14,1,0,0,3,1,0,1,0,0,0
31,0,unknown,1,0,47,0,0,1,1,0,13,1,0,0
36,0,unknown,0,0,15,1,0,5,0,1,0,1,0,0
41,0,unknown,8,17,2,0,0,1,0,3,1,0,0,0
1,0,unknown,7,28,5,5,0,0,1,2,2,0,0,0


In [86]:
maxes = raw_data.max()

In [87]:
maxes

box               24
condition    unknown
doubletap         27
drag             112
draw             355
hold               7
lasso             36
other             28
pinch             13
rev_pinch         14
tap              104
tripletap          6
ui                55
user               9
voice             19
dtype: object

In [88]:
mins = raw_data.min()

In [89]:
mins

box                0
condition    hundred
doubletap          0
drag               0
draw               0
hold               0
lasso              0
other              0
pinch              0
rev_pinch          0
tap                0
tripletap          0
ui                 0
user               1
voice              0
dtype: object

The minimums are all zero, so normalization with (value - min)/(max - min) is the same as value/max...

In [90]:
no_str = raw_data.drop(["condition"], axis = 1)

In [91]:
mins_no_str = mins.drop(["condition"])

In [92]:
maxes_no_str = maxes.drop(["condition"])

In [93]:
no_str.set_index("user")

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,
11,0,0,19,3,1,0,1,1,0,0,0,0,2
26,0,5,25,4,2,0,1,0,4,4,1,0,0
21,0,0,13,3,2,1,1,1,0,2,0,0,0
16,0,0,34,1,0,0,0,0,0,0,0,0,0
46,0,0,14,1,0,0,3,1,0,1,0,0,0
31,0,1,0,47,0,0,1,1,0,13,1,0,0
36,0,0,0,15,1,0,5,0,1,0,1,0,0
41,0,8,17,2,0,0,1,0,3,1,0,0,0
1,0,7,28,5,5,0,0,1,2,2,0,0,0


In [94]:
max_df = pandas.DataFrame(maxes).drop(["condition"])

In [95]:
max_df

,0
box,24
doubletap,27
drag,112
draw,355
hold,7
lasso,36
other,28
pinch,13
rev_pinch,14
tap,104


In [96]:
max_df = max_df.T
max_df.set_index("user")

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,
9,24,27,112,355,7,36,28,13,14,104,6,55,19


In [97]:
max_df = max_df.astype("int32")
max_df

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,user,voice
0,24,27,112,355,7,36,28,13,14,104,6,55,9,19


In [98]:
no_str = no_str.astype("int32")
no_str = no_str.set_index("user") #The assignment is required, this isn't set on an object, but "return result of set()"

In [99]:
normed = no_str/no_str.max()
normed

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,
11,0.000000,0.000000,0.169643,0.008451,0.142857,0.000000,0.035714,0.076923,0.000000,0.000000,0.000000,0.000000,0.105263
26,0.000000,0.185185,0.223214,0.011268,0.285714,0.000000,0.035714,0.000000,0.285714,0.038462,0.166667,0.000000,0.000000
21,0.000000,0.000000,0.116071,0.008451,0.285714,0.027778,0.035714,0.076923,0.000000,0.019231,0.000000,0.000000,0.000000
16,0.000000,0.000000,0.303571,0.002817,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
46,0.000000,0.000000,0.125000,0.002817,0.000000,0.000000,0.107143,0.076923,0.000000,0.009615,0.000000,0.000000,0.000000
31,0.000000,0.037037,0.000000,0.132394,0.000000,0.000000,0.035714,0.076923,0.000000,0.125000,0.166667,0.000000,0.000000
36,0.000000,0.000000,0.000000,0.042254,0.142857,0.000000,0.178571,0.000000,0.071429,0.000000,0.166667,0.000000,0.000000
41,0.000000,0.296296,0.151786,0.005634,0.000000,0.000000,0.035714,0.000000,0.214286,0.009615,0.000000,0.000000,0.000000
1,0.000000,0.259259,0.250000,0.014085,0.714286,0.000000,0.000000,0.076923,0.142857,0.019231,0.000000,0.000000,0.000000


In [100]:
conditions = {}
for condition in adh.conditionMap.keys():
    conditions[condition] = [x for x in range(1,52) if x % 5 == adh.conditionMap[condition] ]
#Now turn it inside out
users=[]
cons=[]
for condition in conditions:
    for user in conditions[condition]:
        users.append(user)
        cons.append(condition)
conditions_df = pandas.DataFrame({"user":users, "condition":cons})
conditions_df.set_index("user")


,condition
user,
1,unknown
6,unknown
11,unknown
16,unknown
21,unknown
26,unknown
31,unknown
36,unknown
41,unknown


In [101]:
normed = normed.join(conditions_df).drop("user", axis=1)

In [102]:
normed["condition"] = normed["condition"].astype("category")

In [103]:
normed

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice,condition
user,,,,,,,,,,,,,,
11,0.000000,0.000000,0.169643,0.008451,0.142857,0.000000,0.035714,0.076923,0.000000,0.000000,0.000000,0.000000,0.105263,thousand
26,0.000000,0.185185,0.223214,0.011268,0.285714,0.000000,0.035714,0.000000,0.285714,0.038462,0.166667,0.000000,0.000000,hundred
21,0.000000,0.000000,0.116071,0.008451,0.285714,0.027778,0.035714,0.076923,0.000000,0.019231,0.000000,0.000000,0.000000,hundred
16,0.000000,0.000000,0.303571,0.002817,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,thousand
46,0.000000,0.000000,0.125000,0.002817,0.000000,0.000000,0.107143,0.076923,0.000000,0.009615,0.000000,0.000000,0.000000,one
31,0.000000,0.037037,0.000000,0.132394,0.000000,0.000000,0.035714,0.076923,0.000000,0.125000,0.166667,0.000000,0.000000,ten
36,0.000000,0.000000,0.000000,0.042254,0.142857,0.000000,0.178571,0.000000,0.071429,0.000000,0.166667,0.000000,0.000000,ten
41,0.000000,0.296296,0.151786,0.005634,0.000000,0.000000,0.035714,0.000000,0.214286,0.009615,0.000000,0.000000,0.000000,one
1,0.000000,0.259259,0.250000,0.014085,0.714286,0.000000,0.000000,0.076923,0.142857,0.019231,0.000000,0.000000,0.000000,unknown


In [104]:
for colname in normed.columns:
    if colname and colname != "condition":
        model = ols('{0} ~ condition'.format(colname), data=normed).fit()
        table = sm.stats.anova_lm(model, typ=1)
        print colname
        print table
        print

box
           df    sum_sq   mean_sq        F    PR(>F)
condition   4  0.090486  0.022622  0.30707  0.871738
Residual   45  3.315104  0.073669      NaN       NaN

doubletap
           df    sum_sq   mean_sq         F    PR(>F)
condition   4  0.084499  0.021125  0.863121  0.493383
Residual   45  1.101372  0.024475       NaN       NaN

drag
           df    sum_sq   mean_sq         F    PR(>F)
condition   4  0.548125  0.137031  3.862703  0.008812
Residual   45  1.596397  0.035475       NaN       NaN

draw
           df    sum_sq   mean_sq        F  PR(>F)
condition   4  0.192455  0.048114  2.73245  0.0405
Residual   45  0.792371  0.017608      NaN     NaN

hold
           df    sum_sq   mean_sq         F    PR(>F)
condition   4  0.120000  0.030000  0.462587  0.762765
Residual   45  2.918367  0.064853       NaN       NaN

lasso
           df    sum_sq   mean_sq         F    PR(>F)
condition   4  0.201142  0.050285  0.884874  0.480681
Residual   45  2.557253  0.056828       NaN       NaN


In [105]:
no_str.sum()

box           113
doubletap      95
drag         1084
draw          761
hold           66
lasso         184
other         189
pinch          26
rev_pinch      49
tap           514
tripletap      19
ui            112
voice          44
dtype: int64

In [106]:
no_str.sum().sum()

3256

These give me the total counts of each gesture, and the total counts. 

In [107]:
adh = all_data_handler.UserData()

In [108]:
per_task_df = pandas.DataFrame(adh.toPandas())

In [109]:
per_task_df

,box,condition,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,task,tripletap,ui,user,voice
0,0,one,0,1,0,0,0,0,0,0,0,remove,0,0,42,1
1,0,one,0,1,0,0,0,0,0,0,3,mark,0,0,42,1
2,0,one,0,2,0,0,0,0,0,0,4,move_a,0,0,42,0
3,0,one,1,1,3,2,0,0,0,0,1,stop,0,0,42,0
4,0,one,0,1,0,0,0,0,0,0,0,move_wall,0,0,42,0
5,0,one,0,4,0,0,0,0,0,0,0,divide_color_2,0,0,42,0
6,0,one,0,4,1,0,0,0,0,0,0,divide_color_1,1,0,42,0
7,0,one,1,2,0,0,0,0,0,0,4,split,0,0,42,0
8,0,one,0,0,0,0,0,0,0,2,3,divide_color_mix,2,0,42,0
9,0,one,0,1,1,0,0,0,0,0,0,crate_dispersed,0,0,42,0


In [110]:
per_task_df.groupby("task").mean()

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice
task,,,,,,,,,,,,,
crate,0.100,0.200,1.440,1.020,0.160,0.220,0.160,0.000,0.000,0.800,0.060,0.220,0.020
crate_dispersed,0.040,0.160,1.040,1.060,0.120,0.140,0.200,0.000,0.000,0.900,0.060,0.140,0.060
disperse,0.175,0.075,1.325,1.175,0.025,0.175,0.575,0.000,0.400,0.550,0.025,0.125,0.050
divide,0.275,0.025,1.750,0.875,0.000,0.325,0.450,0.000,0.025,0.125,0.025,0.050,0.025
divide_color_1,0.220,0.000,2.660,0.760,0.020,0.340,0.180,0.000,0.060,0.420,0.020,0.020,0.020
divide_color_2,0.220,0.000,1.960,0.540,0.020,0.300,0.280,0.000,0.000,0.340,0.000,0.020,0.020
divide_color_mix,0.080,0.180,2.040,0.640,0.080,0.280,0.220,0.000,0.120,0.640,0.040,0.180,0.140
line,0.175,0.125,1.200,1.400,0.125,0.175,0.200,0.025,0.025,1.100,0.025,0.075,0.125
mark,0.075,0.300,0.400,0.325,0.150,0.225,0.125,0.000,0.100,0.925,0.000,0.175,0.050


In [111]:
per_task_df.groupby("condition").mean()

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice
condition,,,,,,,,,,,,,
hundred,0.377778,0.050000,1.088889,0.538889,0.066667,0.305556,0.366667,0.000000,0.022222,0.472222,0.000000,0.105556,0.127778
one,0.000000,0.327273,0.954545,0.436364,0.154545,0.027273,0.009091,0.000000,0.045455,0.963636,0.036364,0.181818,0.063636
ten,0.144444,0.144444,2.300000,0.538889,0.127778,0.594444,0.188889,0.100000,0.088889,1.316667,0.066667,0.350000,0.005556
thousand,0.105556,0.016667,1.105556,2.333333,0.016667,0.100000,0.405556,0.011111,0.061111,0.172222,0.000000,0.055556,0.061111
unknown,0.000000,0.131250,1.062500,0.618750,0.068750,0.006250,0.093750,0.037500,0.081250,0.343750,0.018750,0.000000,0.012500


In [112]:
col_list = list(per_task_df)

In [113]:
col_list.remove("user")

In [114]:
col_list.remove("condition")

In [115]:
col_list.remove("task")

In [116]:
df[col_list].sum(axis=1)

0      27
1      46
2      23
3      35
4      20
5      64
6      23
7      32
8      50
9      76
10     31
11     58
12     37
13     42
14     39
15     99
16     35
17     27
18    361
19     71
20    119
21     52
22     68
23     60
24    115
25     40
26     64
27     36
28     55
29     25
30     91
31     46
32    117
33    157
34     99
35    100
36    190
37     56
38    113
39    105
40     18
41     16
42     24
43     27
44     49
45     39
46     28
47     31
48    106
49     14
dtype: int64

This gets me a list of how many gestures a user made, since it's on the summed user data

In [117]:
per_task_df[col_list].sum(axis=1)

0       2
1       5
2       6
3       8
4       1
5       4
6       6
7       7
8       7
9       2
10      1
11      5
12     12
13     13
14      6
15      9
16      3
17      2
18      1
19     13
20      9
21      1
22      6
23      5
24      4
25      9
26      5
27     11
28      3
29      9
       ..
780     4
781    37
782    19
783     4
784     5
785     1
786     3
787    10
788     3
789     7
790     8
791     9
792    13
793     2
794     5
795     9
796     3
797     6
798     9
799     4
800     9
801     6
802     5
803     4
804     4
805     5
806     5
807     4
808     4
809     2
dtype: int64

This gets me per task gestures, so I can add it to my existing data frame like so

In [118]:
per_task_df["total_gestures"] = per_task_df[col_list].sum(axis=1)

In [119]:
per_task_df

,box,condition,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,task,tripletap,ui,user,voice,total_gestures
0,0,one,0,1,0,0,0,0,0,0,0,remove,0,0,42,1,2
1,0,one,0,1,0,0,0,0,0,0,3,mark,0,0,42,1,5
2,0,one,0,2,0,0,0,0,0,0,4,move_a,0,0,42,0,6
3,0,one,1,1,3,2,0,0,0,0,1,stop,0,0,42,0,8
4,0,one,0,1,0,0,0,0,0,0,0,move_wall,0,0,42,0,1
5,0,one,0,4,0,0,0,0,0,0,0,divide_color_2,0,0,42,0,4
6,0,one,0,4,1,0,0,0,0,0,0,divide_color_1,1,0,42,0,6
7,0,one,1,2,0,0,0,0,0,0,4,split,0,0,42,0,7
8,0,one,0,0,0,0,0,0,0,2,3,divide_color_mix,2,0,42,0,7
9,0,one,0,1,1,0,0,0,0,0,0,crate_dispersed,0,0,42,0,2


The normalization that Mark used, dividing the count of each gesture by the total gestures used on a task, makes more sense than dividing by total gestures done by that user, as it converts each gesture into a proportion within that task, which permits comparisons across tasks, even when a user used many more gestures for the task. 

For example, if a user in the 10-robot case used individual moves for each of the robots in the "move_a" task, and a user in the one-robot case also used individual moves for each of the robots, then the proportions will be the same, 10/10 and 1/1, or 1, respetively. 

Part of the value of this is that I can then do e.g. t-tests to see if the means between conditions are different for a given gesture. My expectation if selection gestures are mainly used in the 10 and 100 case is that those means will be different (reject the null hypothesis) and the means in the 1, 1000, and unknown cases will not be different (accept the null hypothesis). 

Each user, in this case, is a sample, drawn from the vast population of possible users. 

In [120]:
col_list
per_task_df[col_list] = per_task_df[col_list].div(per_task_df.total_gestures, axis=0)

In [121]:
per_task_df

,box,condition,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,task,tripletap,ui,user,voice,total_gestures
0,0.000000,one,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,remove,0.000000,0.000000,42,0.5,2
1,0.000000,one,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.600000,mark,0.000000,0.000000,42,0.2,5
2,0.000000,one,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.666667,move_a,0.000000,0.000000,42,0.0,6
3,0.000000,one,0.125000,0.125000,0.375000,0.250000,0.000000,0.000000,0,0.000000,0.125000,stop,0.000000,0.000000,42,0.0,8
4,0.000000,one,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,move_wall,0.000000,0.000000,42,0.0,1
5,0.000000,one,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,divide_color_2,0.000000,0.000000,42,0.0,4
6,0.000000,one,0.000000,0.666667,0.166667,0.000000,0.000000,0.000000,0,0.000000,0.000000,divide_color_1,0.166667,0.000000,42,0.0,6
7,0.000000,one,0.142857,0.285714,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.571429,split,0.000000,0.000000,42,0.0,7
8,0.000000,one,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.285714,0.428571,divide_color_mix,0.285714,0.000000,42,0.0,7
9,0.000000,one,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000,0,0.000000,0.000000,crate_dispersed,0.000000,0.000000,42,0.0,2


In [122]:
per_task_df.groupby("condition").mean()

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice,total_gestures
condition,,,,,,,,,,,,,,
hundred,0.110341,0.020904,0.257534,0.171224,0.019240,0.068516,0.127448,0.000000,0.010358,0.093717,0.000000,0.019564,0.101152,3.522222
one,0.000000,0.041150,0.481812,0.153971,0.038474,0.023148,0.001029,0.000000,0.010362,0.179213,0.007275,0.043658,0.019907,3.200000
ten,0.029077,0.034595,0.344130,0.162049,0.025498,0.113012,0.070926,0.010195,0.014220,0.129350,0.007637,0.058200,0.001111,5.966667
thousand,0.024571,0.011173,0.374212,0.253558,0.008101,0.031260,0.178039,0.008380,0.021947,0.040714,0.000000,0.020112,0.027933,4.444444
unknown,0.000000,0.050652,0.475144,0.208493,0.027813,0.006369,0.075372,0.033970,0.031900,0.070117,0.010616,0.000000,0.009554,2.475000


In [123]:
per_task_df.groupby("task").mean()

,box,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice,total_gestures
task,,,,,,,,,,,,,,
crate,0.019222,0.033222,0.372433,0.193688,0.039205,0.064359,0.063556,0.000000,0.000000,0.126238,0.009744,0.058333,0.020000,4.400
crate_dispersed,0.006771,0.035417,0.409191,0.213003,0.020023,0.034226,0.059722,0.000000,0.000000,0.148699,0.006232,0.035962,0.030754,3.920
disperse,0.054167,0.019643,0.197631,0.114265,0.008333,0.038333,0.308333,0.000000,0.093750,0.050962,0.025000,0.052083,0.037500,4.675
divide,0.059048,0.004167,0.480476,0.147917,0.000000,0.081250,0.145000,0.000000,0.012500,0.033333,0.004167,0.007143,0.025000,3.950
divide_color_1,0.040816,0.000000,0.569606,0.158503,0.006803,0.064796,0.067055,0.000000,0.008746,0.057823,0.003401,0.002041,0.020408,4.720
divide_color_2,0.044558,0.000000,0.560544,0.129252,0.006803,0.068537,0.102041,0.000000,0.000000,0.065306,0.000000,0.002551,0.020408,3.700
divide_color_mix,0.015986,0.030207,0.443427,0.152000,0.009038,0.058047,0.081633,0.000000,0.026239,0.102195,0.005831,0.031519,0.043878,4.640
line,0.065000,0.033333,0.248873,0.286797,0.013889,0.019716,0.087500,0.002273,0.012500,0.116786,0.008333,0.021667,0.083333,4.775
mark,0.020833,0.074342,0.202917,0.119768,0.096875,0.090476,0.050000,0.000000,0.026488,0.239068,0.000000,0.049232,0.030000,2.850


In [124]:
col_list

['box',
 'doubletap',
 'drag',
 'draw',
 'hold',
 'lasso',
 'other',
 'pinch',
 'rev_pinch',
 'tap',
 'tripletap',
 'ui',
 'voice']

In [125]:
per_task_df.groupby(["condition","task"]).mean()

box  doubletap      drag      draw      hold  \
condition task                                                                  
hundred   crate             0.078333   0.000000  0.290476  0.250000  0.025000   
          crate_dispersed   0.012500   0.020000  0.272619  0.170000  0.045000   
          disperse          0.191667   0.000000  0.120000  0.160000  0.000000   
          divide            0.133333   0.000000  0.400000  0.100000  0.000000   
          divide_color_1    0.111111   0.000000  0.314815  0.111111  0.000000   
          divide_color_2    0.135000   0.000000  0.290000  0.100000  0.000000   
          divide_color_mix  0.078333   0.050000  0.284615  0.033333  0.000000   
          line              0.160000   0.000000  0.258824  0.279412  0.000000   
          mark              0.000000   0.100000  0.100000  0.133333  0.100000   
          merge             0.194444   0.000000  0.100000  0.116667  0.061111   
          move_a            0.100000   0.000000  0.450000  0.180000  0.025000   
          move_wall         0.108333   0.000000  0.454167  0.200000  0.000000   
          patrol_a          0.135873   0.000000  0.401455  0.188889  0.000000   
          patrol_screen     0.133333   0.000000  0.291667  0.283333  0.000000   
          remove            0.040000   0.066667  0.164444  0.216667  0.084444   
          split             0.128307   0.000000  0.142857  0.101852  0.000000   
          square            0.100000   0.033333  0.158333  0.346250  0.000000   
          stop              0.150000   0.100000  0.150000  0.100000  0.000000   
one       crate             0.000000   0.044444  0.000000  0.122222  0.116667   
          crate_dispersed   0.000000   0.055556  0.333333  0.247222  0.022222   
          divide_color_1    0.000000   0.000000  0.733333  0.126667  0.033333   
          divide_color_2    0.000000   0.000000  0.800000  0.050000  0.033333   
          divide_color_mix  0.000000   0.086905  0.245000  0.298333  0.044286   
          mark              0.000000   0.047368  0.545000  0.055263  0.037500   
          move_a            0.000000   0.000000  0.666667  0.175000  0.000000   
          move_wall         0.000000   0.000000  0.800000  0.200000  0.000000   
          remove            0.000000   0.021021  0.513889  0.117117  0.006006   
          split             0.000000   0.067619  0.538571  0.050000  0.000000   
          stop              0.000000   0.129167  0.112500  0.257500  0.125000   
ten       crate             0.006667   0.055000  0.360577  0.208718  0.021026   
...                              ...        ...       ...       ...       ...   
thousand  divide_color_1    0.050000   0.000000  0.578571  0.200000  0.000000   
          divide_color_2    0.050000   0.000000  0.483333  0.233333  0.000000   
          divide_color_mix  0.000000   0.000000  0.506667  0.157576  0.000000   
          line              0.050000   0.000000  0.283333  0.366667  0.000000   
          mark              0.033333   0.050000  0.066667  0.133333  0.100000   
          merge             0.050000   0.000000  0.125000  0.200000  0.025000   
          move_a            0.000000   0.000000  0.533333  0.163218  0.000000   
          move_wall         0.000000   0.000000  0.375000  0.200000  0.000000   
          patrol_a          0.000000   0.000000  0.541667  0.237500  0.000000   
          patrol_screen     0.050000   0.000000  0.533333  0.200000  0.000000   
          remove            0.000000   0.055556  0.388889  0.166667  0.000000   
          split             0.050000   0.000000  0.150000  0.400000  0.000000   
          square            0.015385   0.000000  0.030769  0.708333  0.000000   
          stop              0.000000   0.100000  0.430000  0.150000  0.000000   
unknown   crate             0.000000   0.066667  0.633333  0.137500  0.033333   
          crate_dispersed   0.000000   0.111111  0.620370  0.166667  0.000000   
          disperse          0.000000   0.078571  0.171429  0.100

In [126]:
with_idx = per_task_df.set_index(["condition","task"])

In [127]:
with_idx.loc[("one", "move_a")].mean()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


box               0.000000e+00
doubletap         0.000000e+00
drag              6.666667e-01
draw              1.750000e-01
hold              0.000000e+00
lasso             0.000000e+00
other             0.000000e+00
pinch             0.000000e+00
rev_pinch         0.000000e+00
tap               1.583333e-01
tripletap         0.000000e+00
ui                0.000000e+00
user              4.227225e+16
voice             0.000000e+00
total_gestures    2.100000e+00
dtype: float64

In [128]:
with_idx.loc[("one", "crate")].mean()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


box               0.000000e+00
doubletap         4.444444e-02
drag              0.000000e+00
draw              1.222222e-01
hold              1.166667e-01
lasso             2.000000e-01
other             1.111111e-02
pinch             0.000000e+00
rev_pinch         0.000000e+00
tap               2.388889e-01
tripletap         3.333333e-02
ui                2.333333e-01
user              4.227225e+16
voice             0.000000e+00
total_gestures    2.500000e+00
dtype: float64

In [129]:
stats.f_oneway(with_idx.loc[("one", "crate")], with_idx.loc[("one", "move_a")])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


F_onewayResult(statistic=array([  0.00000000e+00,   1.00000000e+00,   2.25000000e+01,
         1.16264090e-01,   2.19402985e+00,   2.25000000e+00,
         1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         3.53608970e-01,   1.00000000e+00,   3.64462810e+00,
         2.38122249e-14,   0.00000000e+00,   1.76904177e-01]), pvalue=array([  1.00000000e+00,   3.30564931e-01,   1.62270017e-04,
         7.37069942e-01,   1.55839851e-01,   1.50950452e-01,
         3.30564931e-01,   1.00000000e+00,   1.00000000e+00,
         5.59478654e-01,   3.30564931e-01,   7.23238745e-02,
         9.99999878e-01,   1.00000000e+00,   6.79026090e-01]))

In [130]:
stats.f_oneway(with_idx.loc[("one", "crate")], with_idx.loc[("unknown", "crate")])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


F_onewayResult(statistic=array([  3.60000000e+01,   1.08108108e-01,   2.25625000e+01,
         1.26767941e-02,   9.49367089e-01,   2.25000000e+00,
         1.00000000e+00,   3.60000000e+01,   3.60000000e+01,
         7.60510175e-01,   1.00000000e+00,   3.64462810e+00,
         2.18181818e-02,   3.60000000e+01,   9.55832162e-01]), pvalue=array([  1.12699711e-05,   7.46102928e-01,   1.59950179e-04,
         9.11600763e-01,   3.42793802e-01,   1.50950452e-01,
         3.30564931e-01,   1.12699711e-05,   1.12699711e-05,
         3.94652878e-01,   3.30564931e-01,   7.23238745e-02,
         8.84213818e-01,   1.12699711e-05,   3.41197055e-01]))

I'm still not sure how to interpret these. I think that the p-values are how likely each gesture has the same population mean, or rather, how much the null hypothesis that they have the same population mean is to be correct. Assuming the arrays are in the same order as they are in the data table, they are "box, doubletap, drag, draw, hold, lasso, other, pinch, rev_pinch, tap, tripletap, ui, voice, total_gestures", so drag is the only one that isn't likely from the same population mean. Since the one and unknown conditions are kind of similar, in that there is only one object represented, this isn't too surprising.  

The problem with this is that the p-values array has 15 entries for 14 columns...

In [131]:
stats.f_oneway(with_idx.loc[("one", "crate")], with_idx.loc[("ten", "crate")])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


F_onewayResult(statistic=array([ 1.        ,  0.03144325,  8.07396237,  0.34368817,  1.42527327,
        0.94785334,  0.7804878 ,         nan,         nan,  0.77043191,
        0.23902439,  2.49230769,  0.02181818,         nan,  9.0452072 ]), pvalue=array([ 0.33056493,  0.86123523,  0.01082856,  0.56498798,  0.24803442,
        0.3431692 ,  0.38863943,         nan,         nan,  0.391649  ,
        0.63081535,  0.13181603,  0.88421382,         nan,  0.00756064]))

Going with the above interpretation, the only statistically significant difference here is the number of gestures used?

In [132]:
stats.f_oneway(with_idx.loc[("one", "crate")].mean(), with_idx.loc[("ten", "crate")].mean())

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


F_onewayResult(statistic=0.0091821839386080836, pvalue=0.92434312689759945)

I'm not sure what the means would mean in this case, since they would be means of different gestures, getting treated like the same kind of data. Maybe cutting the data down to just be e.g. use of lasso would help. 

In [133]:
stats.f_oneway(with_idx.loc[("one", "crate")]["lasso"], with_idx.loc[("ten", "crate")]["lasso"])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


F_onewayResult(statistic=0.94785333751175194, pvalue=0.34316920312272808)

Ok, so that's the p-value that is in the 6th position in the above sets, which were not cut down to just lasso, so it probably is the probablility that the lasso instances were drawn from different population means. It's also not < 0.05, so it's not really all that great a likelyhood. 

In [134]:
stats.f_oneway(with_idx.loc[("one", "crate")]["lasso"], with_idx.loc[("hundred", "crate")]["lasso"])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


F_onewayResult(statistic=1.0383233532934133, pvalue=0.32171309080058691)

In [136]:
# for cond in with_idx.index.levels[0]:
#     for task in with_idx.index.levels[1]:
#         res = stats.f_oneway(with_idx.loc[(cond, task)], with_idx.loc[(cond, task)])
#         print cond, task, res.statistic, res.pvalue
#Fails because some conditions don't have some tasks

In [137]:
model = statsmodels.formula.api.ols('lasso ~ condition + task', data=per_task_df).fit()

In [138]:
table = sm.stats.anova_lm(model, typ=1)

AttributeError: 'DataFrame' object has no attribute 'design_info'